# NER + CRF

モデルのCRFレイヤも学習させてみている

In [ ]:
from comet_ml import Experiment
import os
import re
import gc
import json
import time
import glob
import pickle
import string
import random
import itertools
import warnings
from collections import defaultdict, Counter
from dotenv import load_dotenv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

import nltk
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.sequence import pad_sequences

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import CometLogger
from torch_optimizer import RAdam
from torchcrf import CRF
import transformers
from transformers import BertModel, BertForTokenClassification, RobertaTokenizer, RobertaForTokenClassification

%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', 300)
pd.options.display.float_format = '{:.5f}'.format

load_dotenv('.env')

## Utils

In [ ]:
def preprocess_text(text: str) -> str:
    """
    テキストの前処理　クリーニング
    """
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    
    return text


def select_predict_section(train, ratio=100):
    """
    trainデータで、正解ラベルが出現しているセクションのリストを返却する
    ratioに設定した割合で、trainでの出現割合の高いセクションを選択する. すべて抽出したい場合は100を指定する
    """
    _df = pd.DataFrame(train['section_title'].value_counts()).reset_index().rename(columns={'index':'section', 'section_title':'section_count'})
    # 全体件数から算出した比率
    _df['ratio'] =  _df['section_count'].apply(lambda x: (x / train.shape[0]) * 100)
    # 累積和
    _df['ratio_cum_sum'] =  _df['ratio'].cumsum()
    
    # sectionの抽出
    section_list = _df[_df['ratio_cum_sum'] <= ratio]['section'].unique().tolist()
    
    return section_list


def expand_data(df, max_len, override=0, label_name=None) -> pd.DataFrame:
    """
    指定したmax_lenを超えるテキストに対して分割を行う関数
    
    ---------------------------------------
    Parameters
    
    df: pd.DataFrame
        拡張対象のデータフレーム
        pub_id, section_title, textが存在していること
    max_len: int
        分割する単語数
    override: int
        分割する際に重複する単語数
    label_name: str
        分割するlabel名
        
    ---------------------------------------
    Returns
    
    res: pd.DataFrame
        分割したテキストで構成されたデータフレーム
    
    """
    # 結果格納用データフレーム
    res = pd.DataFrame()
    # テキストの前処理
    df['text_clean'] = df['text'].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)
    
    ids = df['pub_id'].unique()
    
    for _id in ids:   
        tar = df[df['pub_id'] == _id]

        for i in range(len(tar)):
            row = tar.iloc[i]
            tar_text = row['text_clean'].split(' ')
            text_len = len(tar_text)

            # 単語数がmax_lenより小さい場合はそのまま
            if text_len <= max_len:
                res = pd.concat([res, pd.DataFrame(row).T], axis=0)  # Bug Fix

            # 単語数がmax_lenより大きい場合は分割する
            elif text_len > max_len:
                # 分割する数を計算する
                num_divide = int(np.ceil(text_len / (max_len - override)))
                # 分割する分行を複製しておく（データフレーム化）
                tmp_df = pd.DataFrame([row] * num_divide)
                # 分割後のテキストを格納しておくリスト
                divided_texts = []
                divided_labels = []

                # max_lenごとのテキストに分割する
                for i in range(len(tmp_df)):
                    div_text = tar_text[int(i * (max_len - override)) : int(i * (max_len - override) + max_len)]
                    # リストから文字列に直す
                    div_text = ' '.join(div_text)
                    # 結果を一旦リストにまとめておく
                    divided_texts.append(div_text)
                    
                    # ラベルを分割する場合
                    # TODO 'target_labels'も正しく変換する
                    if label_name is not None:
                        div_label = row[label_name][int(i * (max_len - override)) : int(i * (max_len - override) + max_len)]
                        divided_labels.append(div_label)
                    

                # 複製しておいたデータフレームに置換
                tmp_df['text_clean'] = divided_texts
                if label_name is not None:
                    tmp_df[label_name] = divided_labels
                
                # 全体のデータフレームに結合
                res = pd.concat([res, tmp_df], axis=0)
                
    # 余計な列を削除する
    res = res.dropna()
    res = res.reset_index(drop=True)
    
    return res

# GroupKFold
def get_index_groupk_fold_shuffle(train, group_col, n_splits, i_fold, seed) -> np.array:
    """GroupKFold（shuffleバージョン） クロスバリデーションでのfoldを指定して対応するレコードのインデックスを返す
    """
    # 学習データ・バリデーションデータを分けるインデックスを返す
    # scikit-learnのGroupKFoldはshuffleできない
    group_data = train[group_col]
    unique_group_data = group_data.unique()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    group_k_list = []

    for tr_group_idx, va_group_idx in kf.split(unique_group_data):
        tr_group = unique_group_data[tr_group_idx]
        va_group = unique_group_data[va_group_idx]

        is_tr = pd.DataFrame(group_data.isin(tr_group))
        is_va = pd.DataFrame(group_data.isin(va_group))

        tr_idx = list(is_tr[is_tr[group_col] == True].index)
        va_idx = list(is_va[is_va[group_col] == True].index)

        temp_list = (tr_idx, va_idx)
        group_k_list.append(temp_list)

    return group_k_list[i_fold]


# StratifiedGroupKFold
# hhttps://www.kaggle.com/realsid/stratifiedgroupkfold-using-sklearn

from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits
from sklearn.utils.validation import check_random_state, column_or_1d
from sklearn.utils.multiclass import type_of_target
from collections import defaultdict

class StratifiedGroupKFold(_BaseKFold):
    """Stratified K-Folds iterator variant with non-overlapping groups.
    This cross-validation object is a variation of StratifiedKFold attempts to
    return stratified folds with non-overlapping groups. The folds are made by
    preserving the percentage of samples for each class.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    The difference between GroupKFold and StratifiedGroupKFold is that
    the former attempts to create balanced folds such that the number of
    distinct groups is approximately the same in each fold, whereas
    StratifiedGroupKFold attempts to create folds which preserve the
    percentage of samples for each class as much as possible given the
    constraint of non-overlapping groups between splits.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.
    shuffle : bool, default=False
        Whether to shuffle each class's samples before splitting into batches.
        Note that the samples within each split will not be shuffled.
        This implementation can only shuffle groups that have approximately the
        same y distribution, no global shuffle will be performed.
    random_state : int or RandomState instance, default=None
        When `shuffle` is True, `random_state` affects the ordering of the
        indices, which controls the randomness of each fold for each class.
        Otherwise, leave `random_state` as `None`.
        Pass an int for reproducible output across multiple function calls.
        See :term:`Glossary <random_state>`.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import StratifiedGroupKFold
    >>> X = np.ones((17, 2))
    >>> y = np.array([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    >>> groups = np.array([1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 7, 8, 8])
    >>> cv = StratifiedGroupKFold(n_splits=3)
    >>> for train_idxs, test_idxs in cv.split(X, y, groups):
    ...     print("TRAIN:", groups[train_idxs])
    ...     print("      ", y[train_idxs])
    ...     print(" TEST:", groups[test_idxs])
    ...     print("      ", y[test_idxs])
    TRAIN: [1 1 2 2 4 5 5 5 5 8 8]
           [0 0 1 1 1 0 0 0 0 0 0]
     TEST: [3 3 3 6 6 7]
           [1 1 1 0 0 0]
    TRAIN: [3 3 3 4 5 5 5 5 6 6 7]
           [1 1 1 1 0 0 0 0 0 0 0]
     TEST: [1 1 2 2 8 8]
           [0 0 1 1 0 0]
    TRAIN: [1 1 2 2 3 3 3 6 6 7 8 8]
           [0 0 1 1 1 1 1 0 0 0 0 0]
     TEST: [4 5 5 5 5]
           [1 0 0 0 0]
    Notes
    -----
    The implementation is designed to:
    * Mimic the behavior of StratifiedKFold as much as possible for trivial
      groups (e.g. when each group contains only one sample).
    * Be invariant to class label: relabelling ``y = ["Happy", "Sad"]`` to
      ``y = [1, 0]`` should not change the indices generated.
    * Stratify based on samples as much as possible while keeping
      non-overlapping groups constraint. That means that in some cases when
      there is a small number of groups containing a large number of samples
      the stratification will not be possible and the behavior will be close
      to GroupKFold.
    See also
    --------
    StratifiedKFold: Takes class information into account to build folds which
        retain class distributions (for binary or multiclass classification
        tasks).
    GroupKFold: K-fold iterator variant with non-overlapping groups.
    """

    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle,
                         random_state=random_state)

    def _iter_test_indices(self, X, y, groups):
        # Implementation is based on this kaggle kernel:
        # https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
        # and is a subject to Apache 2.0 License. You may obtain a copy of the
        # License at http://www.apache.org/licenses/LICENSE-2.0
        # Changelist:
        # - Refactored function to a class following scikit-learn KFold
        #   interface.
        # - Added heuristic for assigning group to the least populated fold in
        #   cases when all other criteria are equal
        # - Swtch from using python ``Counter`` to ``np.unique`` to get class
        #   distribution
        # - Added scikit-learn checks for input: checking that target is binary
        #   or multiclass, checking passed random state, checking that number
        #   of splits is less than number of members in each class, checking
        #   that least populated class has more members than there are splits.
        rng = check_random_state(self.random_state)
        y = np.asarray(y)
        type_of_target_y = type_of_target(y)
        allowed_target_types = ('binary', 'multiclass')
        if type_of_target_y not in allowed_target_types:
            raise ValueError(
                'Supported target types are: {}. Got {!r} instead.'.format(
                    allowed_target_types, type_of_target_y))

        y = column_or_1d(y)
        _, y_inv, y_cnt = np.unique(y, return_inverse=True, return_counts=True)
        if np.all(self.n_splits > y_cnt):
            raise ValueError("n_splits=%d cannot be greater than the"
                             " number of members in each class."
                             % (self.n_splits))
        n_smallest_class = np.min(y_cnt)
        if self.n_splits > n_smallest_class:
            warnings.warn(("The least populated class in y has only %d"
                           " members, which is less than n_splits=%d."
                           % (n_smallest_class, self.n_splits)), UserWarning)
        n_classes = len(y_cnt)
        
        
        _, groups_inv, groups_cnt = np.unique(
            groups, return_inverse=True, return_counts=True)
        y_counts_per_group = np.zeros((len(groups_cnt), n_classes))
        for class_idx, group_idx in zip(y_inv, groups_inv):
            y_counts_per_group[group_idx, class_idx] += 1

        y_counts_per_fold = np.zeros((self.n_splits, n_classes))
        groups_per_fold = defaultdict(set)

        if self.shuffle:
            rng.shuffle(y_counts_per_group)

        # Stable sort to keep shuffled order for groups with the same
        # class distribution variance
        sorted_groups_idx = np.argsort(-np.std(y_counts_per_group, axis=1),
                                       kind='mergesort')

        for group_idx in sorted_groups_idx:
            group_y_counts = y_counts_per_group[group_idx]
            best_fold = self._find_best_fold(
                y_counts_per_fold=y_counts_per_fold, y_cnt=y_cnt,
                group_y_counts=group_y_counts)
            y_counts_per_fold[best_fold] += group_y_counts
            groups_per_fold[best_fold].add(group_idx)

        for i in range(self.n_splits):
            test_indices = [idx for idx, group_idx in enumerate(groups_inv)
                            if group_idx in groups_per_fold[i]]
            yield test_indices

    def _find_best_fold(
            self, y_counts_per_fold, y_cnt, group_y_counts):
        best_fold = None
        min_eval = np.inf
        min_samples_in_fold = np.inf
        for i in range(self.n_splits):
            y_counts_per_fold[i] += group_y_counts
            # Summarise the distribution over classes in each proposed fold
            std_per_class = np.std(
                y_counts_per_fold / y_cnt.reshape(1, -1),
                axis=0)
            y_counts_per_fold[i] -= group_y_counts
            fold_eval = np.mean(std_per_class)
            samples_in_fold = np.sum(y_counts_per_fold[i])
            is_current_fold_better = (
                fold_eval < min_eval or
                np.isclose(fold_eval, min_eval)
                and samples_in_fold < min_samples_in_fold
            )
            if is_current_fold_better:
                min_eval = fold_eval
                min_samples_in_fold = samples_in_fold
                best_fold = i
        return best_fold

In [ ]:
# SAM Optimizer
# https://github.com/davda54/sam/blob/main/sam.py

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, adaptive=False, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, adaptive=adaptive, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = (torch.pow(p, 2) if group["adaptive"] else 1.0) * p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        ((torch.abs(p) if group["adaptive"] else 1.0) * p.grad).norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

## Load Data

In [ ]:
def load_data(data_dir, expand=False):
    # takapyさんのNERDatasetを読み込む
    TRAIN_FOR_NER_PATH = '../input/train_0528.pkl'
    with open(TRAIN_FOR_NER_PATH, 'rb') as f:
        train = pickle.load(f)
    
    # Testデータの読み込み
    test_files = glob.glob(data_dir + "test/*.json")

    test = pd.DataFrame()

    # jsonからDataFrameに
    for tar in test_files:
        file_data = pd.read_json(tar)
        file_data.insert(0,'pub_id', tar.split('/')[-1].split('.')[0])
        test = pd.concat([test, file_data])
    

    # testデータのIdはデータフレームにまとめておく
    test_ids = pd.DataFrame({
        'Id': test['pub_id'].unique()
    })
    
    # testデータについて特定のセクションタイトルに絞る
    section_list = select_predict_section(train, 100)
    test = test[test['section_title'].isin(section_list)].reset_index(drop=True)
    
    if expand:
#         train = expand_data(train, max_len=512, override=10, label_name='ner_label_bioes')
        test = expand_data(test, max_len=512, override=10)
    
    return train, test, test_ids

## Dataset

Pytorch Datasetを定義する

In [ ]:
class ColeridgeDataset(Dataset):
    def __init__(self, df, tokenizer, cfg, max_len=512, phase='train'):
        self.df = df
        self.tokenizer = tokenizer
        self.cfg = cfg
        self.max_len = max_len
        self.phase = phase
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Datasetの返り値を指定する
        # トークナイズされたテキストID [list(int)]
        input_ids = row['input_ids']
        # トークナイズされたラベルID [list(int)]
        label = row['tokenized_label'] if self.phase != 'test' else None
        # トークナイズ前のテキスト [List(str)]
        text = row['text_clean_list']
        # 対象のpub_id [str]
        pub_id = row['pub_id']
        # CVスコアを計算するための正解ラベル [list(str)]
        label_str = row['target_labels'] if self.phase != 'test' else None
        
        # B-NER, S-NERが存在するようにoffset分ずらす
        if self.phase == 'train':
            b_ner_list = [n for n, v in enumerate(row[self.cfg.ner_label_name]) if v in ['B-NER', 'S-NER']]
            # B-NERが位置する最初のインデックスがmax_lenより大きい場合（単純にpaddingしてしまうとラベルが消える）
            if np.min(b_ner_list) > self.cfg.max_len:
                # offsetは乱数で指定
                offset = random.randint(int(np.min(b_ner_list) - self.cfg.max_len), int(len(input_ids) - self.cfg.max_len))
                input_ids = input_ids[offset:]
                label = label[offset:]
                text = text[offset:]
        
        return input_ids, label, text, pub_id, label_str

## DataModule

Lightningのデータモジュールを作成する

学習データのセットアップやデータローダーの生成を行う

In [ ]:
class ColeridgeDataModule(pl.LightningDataModule):
    def __init__(self, train, test, cfg, tokenizer, preprocess_fn):
        """
        ------------------------------------
        Parameters
        train: pd.dataframe
            train dataset
        test: pd.dataframe
            test dataset
        cfg: DictConfig
            Config
        tokenizer:
            Pretrained Tokenizer
            if tokenizer is None, 'nltk tokenizer' is used.
        preprocess_fn: function
            Preprocessing Function str => str
        """
        super(ColeridgeDataModule, self).__init__()
        self.train = train
        self.test = test
        self.cfg = cfg
        self.tokenizer = tokenizer
        self.preprocess_fn = preprocess_fn
        
    def _padding(self, inp, value=0):
        inp = pad_sequences(
            inp, 
            maxlen=self.cfg.max_len,
            dtype="long",
            value=value,
            truncating="post", 
            padding="post"
        )
        
        return inp
        
        
    def train_collate_fn(self, batch):
        """
        バッチをまとめるときに処理する関数
        パディングを実装
        """
        input_ids, label, text, pub_id, label_str = list(zip(*batch))

        # Padding - input_ids
        input_ids = self._padding(input_ids)
        # Attenstion Mask
        attention_mask = [[int(i > 0) for i in ii] for ii in input_ids]

        # Padding - Label
        label = self._padding(label, value=self.tag2idx["PAD"])

        inp = {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'token_type_ids': None,
            'labels': torch.tensor(label, dtype=torch.long)
        }

        return inp, text, pub_id, label_str
        
        
    def test_collate_fn(self, batch):
        """
        バッチをまとめるときに処理する関数
        パディングを実装
        """
        input_ids, label, text, pub_id, label_str = list(zip(*batch))

        # Padding - input_ids
        input_ids = self._padding(input_ids)
        # Attenstion Mask
        attention_mask = [[int(i > 0) for i in ii] for ii in input_ids]
            
        inp = {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'token_type_ids': None
        }

        return inp, text, pub_id, label_str
    

    def prepare_data(self):
        # テキストの前処理
        self.train['text_clean'] = self.train['text'].apply(lambda x: self.preprocess_fn(x) if isinstance(x, str) else x)
        self.train['text_clean_list'] = self.train['text_clean'].apply(lambda x: x.split(' ') if isinstance(x, str) else x)
        self.test['text_clean'] = self.test['text'].apply(lambda x: self.preprocess_fn(x) if isinstance(x, str) else x)
        self.test['text_clean_list'] = self.test['text_clean'].apply(lambda x: x.split(' ') if isinstance(x, str) else x)

        # 単語数による制限を設ける
        self.train['text_len'] = self.train['text_clean_list'].apply(lambda x: len(x))
        self.test['text_len'] = self.test['text_clean_list'].apply(lambda x: len(x))
        self.train = self.train.query("@self.cfg.input_min_len <= text_len <= @self.cfg.input_max_len").reset_index(drop=True)
        self.test = self.test.query("@self.cfg.input_min_len <= text_len <= @self.cfg.input_max_len").reset_index(drop=True)
        
        
        # Tokenize - Text
        def tokenize(text):
            return [self.tokenizer.convert_tokens_to_ids(txt) for txt in text]
        
        self.train['input_ids'] = self.train['text_clean_list'].map(tokenize)
        self.test['input_ids'] = self.test['text_clean_list'].map(tokenize)
        
        # Tokenize - Label
        self.tag2idx = {tag: idx for idx, tag in enumerate(self.cfg.tag_values)}
        self.idx2tag = {idx: tag for tag, idx in self.tag2idx.items()}
        
        def tokenize2(label):
            return [self.tag2idx[l] for l in label]
        
        self.train['tokenized_label'] = self.train[self.cfg.ner_label_name].map(tokenize2)
        
        # Foldの設定
        if self.cfg.cv is not None:
            self.train['fold'] = -1
            self.train['dataset_title_cluster'] = LabelEncoder().fit_transform(self.train['dataset_title_cluster'].values)
            
            if self.cfg.cv == 'groupkfold':
                tr_idx, va_idx = get_index_groupk_fold_shuffle(
                    train=self.train,
                    group_col='dataset_title_cluster',
                    n_splits=self.cfg.n_splits, 
                    i_fold=self.cfg.fold, 
                    seed=self.cfg.seed
                )
                
                self.train.loc[self.train.index[va_idx], 'fold'] = self.cfg.fold                
            
            else:
            # Scikit-learn Fold Split
                for f, (tr_idx, va_idx) in enumerate(self.cfg.cv.split(self.train, 
                                                                       self.train['text_len'].values,
                                                                       self.train['dataset_title_cluster'].values)):
                    self.train.loc[va_idx, 'fold'] = f
                    

    def setup(self, stage=None):
        # Split train valid
        if self.cfg.cv is not None:
            tmp_train = self.train[self.train['fold'] != self.cfg.fold]
            tmp_val = self.train[self.train['fold'] == self.cfg.fold]
        
        else:
            self.train = self.train.sample(frac=1.0).reset_index(drop=True)
            train_data_rate = 0.8
            tmp_train = self.train.loc[:int(len(self.train) * train_data_rate)].reset_index(drop=True)
            tmp_val = self.train.loc[int(len(self.train) * train_data_rate):].reset_index(drop=True)
        
        print('Data Info')
        print(f'Train Dataset: {tmp_train.shape[0]}')
        print(f'Valid Dataset: {tmp_val.shape[0]}')
        print(f'Test  Dataset: {self.test.shape[0]}')
        
        # train, val, testそれぞれのデータセットを作成
        self.train_dataset = ColeridgeDataset(
            tmp_train[['pub_id', 'text_clean_list', 'input_ids', 'tokenized_label', 'target_labels', self.cfg.ner_label_name]],
            tokenizer=self.tokenizer,
            cfg=self.cfg,
            max_len=self.cfg.max_len,
            phase='train'
        )

        self.val_dataset = ColeridgeDataset(
            tmp_val[['pub_id', 'text_clean_list', 'input_ids', 'tokenized_label', 'target_labels', self.cfg.ner_label_name]],
            tokenizer=self.tokenizer,
            cfg=self.cfg,
            max_len=self.cfg.max_len,
            phase='train'
        )

        self.test_dataset = ColeridgeDataset(
            self.test,
            tokenizer=self.tokenizer,
            cfg=self.cfg,
            max_len=self.cfg.max_len,
            phase='test'
        )
        
        del tmp_train, tmp_val
        gc.collect()
        
    # データローダーはそれぞれのメソッドをオーバーライドして定義する
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.cfg.batch_size,
            shuffle=True,
            num_workers=self.cfg.num_workers,
            pin_memory=True,
            collate_fn=self.train_collate_fn
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.cfg.batch_size,
            shuffle=False,
            num_workers=self.cfg.num_workers,
            pin_memory=True,
            collate_fn=self.train_collate_fn
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.cfg.batch_size,
            shuffle=False,
            num_workers=self.cfg.num_workers,
            pin_memory=True,
            collate_fn=self.test_collate_fn
        )

## Model Network

Transformersで展開されている学習済みBERTを使用

https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification

In [ ]:
class BERT_CRF_Model(nn.Module):
    def __init__(self, cfg, num_labels=4):
        super(BERT_CRF_Model, self).__init__()
        self.backbone = transformers.BertModel.from_pretrained(
            cfg.model,
            num_labels=num_labels
        )
        self.dropout = nn.Dropout(cfg.dropout_rate)
        self.position_wise_ff = nn.Linear(768, num_labels)
        self.crf = CRF(num_tags=num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        out = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        out = self.dropout(out[0])
        out = self.position_wise_ff(out)
        
        if labels is not None:
            log_likelihood, sequence_of_tags = self.crf(out, labels, reduction='mean'), self.crf.decode(out)
            # 値がマイナスになるため絶対値を取る
            log_likelihood = torch.abs(log_likelihood)
            return log_likelihood, sequence_of_tags
        else:
            sequence_of_tags = self.crf.decode(out)
            return None, sequence_of_tags
    
    
class RoBERTa_CRF_Model(nn.Module):
    def __init__(self, cfg, num_labels=4):
        super(RoBERTa_CRF_Model, self).__init__()
        self.backbone = transformers.RobertaModel.from_pretrained(cfg.model)
        self.dropout = nn.Dropout(cfg.dropout_rate)
        self.position_wise_ff = nn.Linear(768, num_labels)
        self.crf = CRF(num_tags=num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        out = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        out = self.dropout(out[0])
        out = self.position_wise_ff(out)
        
        if labels is not None:
            log_likelihood, sequence_of_tags = self.crf(out, labels, reduction='mean'), self.crf.decode(out)
            # 値がマイナスになるため絶対値を取る
            log_likelihood = torch.abs(log_likelihood)
            return log_likelihood, sequence_of_tags
        else:
            sequence_of_tags = self.crf.decode(out)
            return None, sequence_of_tags

## Lightning Module

In [ ]:
from typing import List
import numpy as np

def compute_fbeta(y_true: List[List[str]],
                  y_pred: List[List[str]],
                  beta: float = 0.5) -> float:
    """Compute the Jaccard-based micro FBeta score.

    References
    ----------
    - https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/overview/evaluation
    """

    def _jaccard_similarity(str1: str, str2: str) -> float:
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))

    tp = 0  # true positive
    fp = 0  # false positive
    fn = 0  # false negative
    for ground_truth_list, predicted_string_list in zip(y_true, y_pred):
        predicted_string_list_sorted = sorted(predicted_string_list)
        for ground_truth in sorted(ground_truth_list):
            if len(predicted_string_list_sorted) == 0:
                fn += 1
            else:
                similarity_scores = [
                    _jaccard_similarity(ground_truth, predicted_string)
                    for predicted_string in predicted_string_list_sorted
                ]
                matched_idx = np.argmax(similarity_scores)
                predicted_string_list_sorted.pop(matched_idx)
                if similarity_scores[matched_idx] >= 0.5:
                    tp += 1
                else:
                    fp += 1
        fp += len(predicted_string_list_sorted)

    tp *= (1 + beta ** 2)
    fn *= beta ** 2
    fbeta_score = tp / (tp + fp + fn)
    return fbeta_score

In [ ]:
class ColeridgeLightningSystem_CRF(pl.LightningModule):
    def __init__(self, net, cfg, optimizer, scheduler=None):
        """
        ------------------------------------
        Parameters
        net: torch.nn.Module
            Model
        cfg: DictConfig
            Config
        optimizer: torch.optim
            Optimizer
        scheduler: torch.optim.lr_scheduler
            Learning Rate Scheduler
        """
        super(ColeridgeLightningSystem_CRF, self).__init__()
        self.net = net
        self.cfg = cfg
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.best_score = 0
        self.losses = []
        self.scores = []
        self.automatic_optimization = False if cfg.use_sam else True  # SAMを使うときは手動で最適化する

        
    def _get_ner_words_bio(self, text_lists, pred_lists) -> list:
        '''
        予測結果からサブミット用の文字列リストを作成するための関数
        takapyさんのコードを参考
        BIOタグ専用
        -----------------------------------------------
        Parameters
        text_lists: list[list[str]]
            元のテキストデータ
            ex. [[..., 'this', 'model', 'used', ...], [..., 'some', 'data', 'show', ...]]
        pred_lists: list[list[str]]
            予測結果
            ex. [[..., 'O', 'B-NER', 'I-NER', ...], [..., 'B-NER', 'O', 'O', ...]]
            
        -----------------------------------------------
        Returns
        labels: list[str]
            サブミット文字列
            ex. ['data|adni', '', 'noaa']
        
        '''        
        # takapyさんコードを一部拝借：
        labels = []
        
        for text_list, pred_list in zip(text_lists, pred_lists):
            _labels = []
            # B-NERの位置indexを取得. これで予測値にいくつの固有表現が含まれるかを把握
            b_ner_list = [n for n, v in enumerate(pred_list) if v in 'B-NER']

            # 固有表現ラベルが存在する回数ループ
            for ner_label_begin_index in b_ner_list:
                begin_index = ner_label_begin_index  # 固有表現開始位置
                end_index = begin_index  # 固有表現終了位置

                # 固有表現終了位置の算出
                for ner_label in pred_list[begin_index+1:]:
                    # I-NERラベルが終了するまで、end_indexをインクリメントする
                    if ner_label == 'I-NER':
                        end_index += 1
                    else:
                        break

                # begin_index ~ end_index+1までのリストを生成し、そのindexをtextから取得
                get_index = [i for i in range(begin_index, end_index+1)]
                get_list = lambda items, get_indexes: [item for index, item in enumerate(items) if index in get_indexes]
                pred_ner = get_list(text_list, get_index)
                # 文字列に変形
                pred_ner = ' '.join(pred_ner)
                _labels.append(pred_ner)
                
            # 重複を除外&ソート
            _labels = sorted(list(set(_labels)))
            # 予測結果がない場合は''を入れる
            if len(_labels) == 0:
                labels.append('')
            # ちゃんとある場合は'|'で区切る
            else:
                labels.append('|'.join(_labels))
            
        return labels
    
    
    def _get_ner_words_bioes(self, text_lists, pred_lists) -> list:
        '''
        予測結果からサブミット用の文字列リストを作成するための関数
        takapyさんのコードを参考
        BIOESタグ専用
        -----------------------------------------------
        Parameters
        text_lists: list[list[str]]
            元のテキストデータ
            ex. [[..., 'this', 'model', 'used', ...], [..., 'some', 'data', 'show', ...]]
        pred_lists: list[list[str]]
            予測結果
            ex. [[..., 'O', 'B-NER', 'I-NER', ...], [..., 'S-NER', 'O', 'O', ...]]
            
        -----------------------------------------------
        Returns
        labels: list[str]
            サブミット文字列
            ex. ['data|adni', '', 'noaa']
        
        '''
        labels = []
        
        for text_list, pred_list in zip(text_lists, pred_lists):
            _labels = []
            # B-NERの位置indexを取得. これで予測値にいくつの固有表現が含まれるかを把握
            b_ner_list = [n for n, v in enumerate(pred_list) if v in ['B-NER', 'S-NER']]

            # 固有表現ラベルが存在する回数ループ
            for ner_label_begin_index in b_ner_list:
                begin_index = ner_label_begin_index  # 固有表現開始位置
                end_index = begin_index  # 固有表現終了位置

                if pred_list[begin_index:begin_index+1] != 'S-NER':
                    # 固有表現終了位置の算出
                    for ner_label in pred_list[begin_index+1:]:
                        # I-NERラベル、E-NERラベルが終了するまで、end_indexをインクリメントする
                        if ner_label == 'I-NER':
                            end_index += 1
                        elif ner_label == 'E-NER':
                            end_index += 1
                        else:
                            break

                # begin_index ~ end_index+1までのリストを生成
                get_index = [i for i in range(begin_index, end_index+1)]
                get_list = lambda items, get_indexes: [item for index, item in enumerate(items) if index in get_indexes]
                pred_ner = get_list(text_list, get_index)
                # 文字列に変形
                pred_ner = ' '.join(pred_ner)
                _labels.append(pred_ner)

            # 重複を除外&ソート
            _labels = sorted(list(set(_labels)))
            # 予測結果がない場合は''を入れる
            if len(_labels) == 0:
                labels.append('')
            # ちゃんとある場合は'|'で区切る
            else:
                labels.append('|'.join(_labels))

        return labels
        
        
    def _convert_1dim_list(self, outputs, var_name):
        """
        多次元リストを1次元に変換する関数
        """
        res = [x[var_name] for x in outputs]
        res = [list(x) for x in res]
        res = list(itertools.chain.from_iterable(res))

        return res
        
    def configure_optimizers(self):
        """
        活性化関数やスケジューラーを設定する
        """
        if self.scheduler is None:
            return [self.optimizer], []
        else:
            scheduler = {
                'scheduler': self.scheduler,
                'interval': 'step',   # スケジューラーの更新頻度を指定（'step', 'epoch'）
                'frequency': 1
            }

            return [self.optimizer], [scheduler]
        

    def forward(self, inp):
        loss, preds = self.net(**inp)
        return loss, preds


    def training_step(self, batch, batch_idx):
        
        if self.cfg.use_sam:
            inp, _, _, _ = batch
            loss, _ = self.forward(inp)
            
            optimizer = self.optimizers(use_pl_optimizer=True)
            optimizer.zero_grad()
            self.manual_backward(loss)
            optimizer.first_step(zero_grad=True)
            
            loss, _ = self.forward(inp)
            self.manual_backward(loss)
            optimizer.second_step(zero_grad=True)
        
        else:
            inp, _, _, _ = batch
            loss, _ = self.forward(inp)
        self.log(f'train/loss', loss, on_epoch=True)

        del inp
        gc.collect()

        return {'loss': loss}


    def validation_step(self, batch, batch_idx):
        inp, text, pub_id, label_str = batch
        loss, preds = self.forward(inp)
        self.log(f'val/loss', loss, on_epoch=True)

        del inp
        gc.collect()
        
        outputs = {
            'val_loss': loss, 
            'preds': torch.tensor(preds, dtype=torch.long), 
            'texts': text, 
            'pub_ids': pub_id,
            'label_str': label_str
        }
        del loss, preds, text, pub_id, label_str
        gc.collect()

        return outputs


    def validation_epoch_end(self, outputs):
        # 各ステップで記録されたlossなどをまとめる
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        preds = torch.cat([x['preds'] for x in outputs]).detach().cpu().numpy()
        
        # エポックの平均lossを記録
        self.losses.append(avg_loss.item())
        self.log(f'val_epoch/loss', avg_loss, on_step=False, on_epoch=True)
        
        # Text, pub_idを1次元リストに変換
        texts = self._convert_1dim_list(outputs, var_name='texts')
        pub_ids = self._convert_1dim_list(outputs, var_name='pub_ids')
        label_str = self._convert_1dim_list(outputs, var_name='label_str')
        
        # 予測結果（インデックス）から文字列タグに変換
        idx2tag = {idx: tag for idx, tag in enumerate(self.cfg.tag_values)}
        preds = [[idx2tag[idx] for idx in r] for r in preds.tolist()]
        
        # ここまでの結果をDataFrameに保存しておく
        res_df = pd.DataFrame({
            'pub_id': pub_ids,
            'text': texts,
            'pred': preds,
            'label_str': label_str
        })
        
        filename = 'val_pred_epoch_{}.csv'.format(self.current_epoch)
        self.logger.experiment.log_table(filename, res_df)
        
        # submission用のファイルを作る
        if self.cfg.ner_label_name == 'ner_label_bio':
            pred_ner_words = self._get_ner_words_bio(texts, preds)
        elif self.cfg.ner_label_name == 'ner_label_bioes':
            pred_ner_words = self._get_ner_words_bioes(texts, preds)
        
        sub = pd.DataFrame({
            'Id': pub_ids,
            'PredictionString': pred_ner_words
        })
        
        # 先頭や末端に'|'が現れてしまうので削除する
        sub['PredictionString'] = sub['PredictionString'].apply(lambda x: x[1:] if isinstance(x, str) and len(x) > 0 and x[0] == '|' else x)
        sub['PredictionString'] = sub['PredictionString'].apply(lambda x: x[:-1] if isinstance(x, str) and len(x) > 0 and x[-1] == '|' else x)
        
        
        # 予測結果を保存
        filename = 'sub_epoch_{}.csv'.format(self.current_epoch)
        self.logger.experiment.log_table(filename, sub)
        
        
        # Jaccard scoreの計算
        # 予測結果を一旦リストに分割する
        pred_ner_words_lists = [s.split('|') for s in pred_ner_words]
        score = compute_fbeta(y_true=label_str, y_pred=pred_ner_words_lists, beta=0.5)
        self.scores.append(score)
        self.log(f'val_epoch/score', score, on_step=False, on_epoch=True)

        # Save Weights
        filename = 'fold_{}_score_{:.3f}_epoch_{}.pth'.format(self.cfg.fold, score, self.current_epoch)
        torch.save(self.net.state_dict(), filename, _use_new_zipfile_serialization=False)  # https://www.programmersought.com/article/95798060903/
        self.logger.experiment.log_model(name=filename, file_or_folder=filename)
        time.sleep(5)
        os.remove(filename)
        
        # Memory Clear
        del avg_loss, preds, texts, pub_ids, label_str, res_df, sub, pred_ner_words_lists
        gc.collect()
        

        return None


    def test_step(self, batch, batch_idx):
        inp, text, pub_id, _ = batch
        _, preds = self.forward(inp)
        
        outputs = {
            'preds': torch.tensor(preds, dtype=torch.long),
            'pub_ids': pub_id,
            'texts': text
        }

        return outputs


    def test_epoch_end(self, outputs):    
        preds = torch.cat([x['preds'] for x in outputs]).detach().cpu().numpy()
        
        # Text, pub_idを1次元リストに変換
        texts = self._convert_1dim_list(outputs, var_name='texts')
        pub_ids = self._convert_1dim_list(outputs, var_name='pub_ids')
        
        # 予測結果を変換
        idx2tag = {idx: tag for idx, tag in enumerate(self.cfg.tag_values)}
        preds = [[idx2tag[idx] for idx in r] for r in preds.tolist()]
        
        # submission用のファイルを作る
        if self.cfg.ner_label_name == 'ner_label_bio':
            pred_ner_words = self._get_ner_words_bio(texts, preds)
        elif self.cfg.ner_label_name == 'ner_label_bioes':
            pred_ner_words = self._get_ner_words_bioes(texts, preds)
        
        self.sub = pd.DataFrame({
            'Id': pub_ids,
            'PredictionString': pred_ner_words
        })
        
        # TODO 現状ではIdの重複が発生している状態なので、Idをキーとした文字列統合が必要！
        self.sub = self.sub.groupby('Id')['PredictionString'].apply(lambda x: '|'.join(sorted(list(x)))).reset_index()
        # 重複を削除
        self.sub['PredictionString'] = self.sub['PredictionString'].apply(lambda x: '|'.join(sorted(list(set(x.split('|'))))))
        
        # 先頭や末端に'|'が現れてしまうので削除する
        self.sub['PredictionString'] = self.sub['PredictionString'].apply(lambda x: x[1:] if isinstance(x, str) and len(x) > 0 and x[0] == '|' else x)
        self.sub['PredictionString'] = self.sub['PredictionString'].apply(lambda x: x[:-1] if isinstance(x, str) and len(x) > 0 and x[-1] == '|' else x)
        
        filename = 'submission.csv'
        self.logger.experiment.log_table(filename, self.sub)
        
        return None

## Trainer

In [ ]:
# Pretrained

# SciBERT: allenai/scibert_scivocab_uncased
# Biomed RoBERTa: allenai/biomed_roberta_base

In [ ]:
LOGGING = True

def train_fold(cfg):
    # Load Data  --------------------------------------------------------------------------
    data_dir = '../input/'
    train, test, test_ids = load_data(data_dir)

    # Random Seed
    seed_everything(cfg.seed)

    # Logger  --------------------------------------------------------------------------
    if LOGGING:
        logger = CometLogger(
            api_key=os.environ['COMET_ML_API_KEY'],
            project_name=os.environ['COMET_ML_PROJECT_NAME'],
            experiment_name=f"{cfg.model}",
            auto_param_logging=False,
            auto_metric_logging=False
        )

        # コードを出力
        code = ''
        for cell_in in In:
            code += cell_in + '\n'

        logger.experiment.log_code(code=code)

        dict_cfg = {k: v for k, v in dict(cfg.__dict__).items() if '__' not in k}
        logger.log_hyperparams(dict_cfg)
    else:
        logger = None

    # Tokenizer  --------------------------------------------------------------------------
    if 'roberta' in cfg.model:
        tokenizer = transformers.RobertaTokenizer.from_pretrained(cfg.model, do_lower_case=True)
    else:
        tokenizer = transformers.BertTokenizer.from_pretrained(cfg.model, do_lower_case=True)

    # Data Module  --------------------------------------------------------------------------
    dm = ColeridgeDataModule(train, test, cfg, tokenizer, preprocess_fn=preprocess_text)

    # Model Network  --------------------------------------------------------------------------
    if 'roberta' in cfg.model:
        net = RoBERTa_CRF_Model(cfg, num_labels=len(cfg.tag_values))
    else:
        net = BERT_CRF_Model(cfg, num_labels=len(cfg.tag_values))

    # Load Pretrained Weights
    if cfg.pretrained_weight is not None:
        net.load_state_dict(torch.load(cfg.pretrained_weight))

    # Optimizer & Scheduler
    if cfg.optimizer == 'radam':
        if cfg.use_sam:
            optimizer = SAM(net.parameters(), RAdam, lr=cfg.lr, weight_decay=cfg.weight_decay)
        else:
            optimizer = RAdam(net.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    elif cfg.optimizer == 'adam':
        if cfg.use_sam:
            optimizer = SAM(net.parameters(), optim.Adam, lr=cfg.lr, weight_decay=cfg.weight_decay)
        else:
            optimizer = optim.Adam(net.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
            
    scheduler = transformers.get_cosine_with_hard_restarts_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=cfg.warmup_step,
        num_training_steps=cfg.num_step,
        num_cycles=cfg.num_cycles
    )

    # Lightning Module  --------------------------------------------------------------------------
    model = ColeridgeLightningSystem_CRF(net, cfg, optimizer, scheduler)

    # Trainer  --------------------------------------------------------------------------
    epochs = cfg.num_step // cfg.limit_train_batches
    trainer = Trainer(
        logger=logger,
        max_epochs=epochs,
        gpus=-1,
        num_sanity_val_steps=0,
        limit_train_batches=cfg.limit_train_batches,
    #     amp_level='O2',
    #     amp_backend='apex',
        limit_val_batches=200,
    #     fast_dev_run=True
    )

    # Train
    trainer.fit(model, datamodule=dm)

    # Inference
    trainer.test(model, datamodule=dm)
    
    if LOGGING:
        logger.experiment.end()
        
    # Clear Memory
    del train, test, test_ids, tokenizer, dm, net, optimizer, scheduler, model, trainer
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
class cfg:
    max_len = 512
    ner_label_name = 'ner_label_bioes'  # 学習時の正解ラベルの列名
    input_max_len = 2000  # 学習や推論に使用する最大テキスト数
    input_min_len = 32    # 学習や推論に使用する最小テキスト数
    n_splits = 5
    fold = 0
    batch_size = 4
    num_workers = 8
    dropout_rate = 0.5
    optimizer = 'radam'  # 'adam', 'radam'
    use_sam = False      # SAMを使うかどうか
    lr = 1e-4
    weight_decay = 1e-5
    num_step = 3000            #  学習を行うトータルのステップ数
    limit_train_batches = 200  # 指定したステップごとにValを行う
    model = 'allenai/biomed_roberta_base'   # memo: SciroBERTa: https://huggingface.co/allenai/biomed_roberta_base
    warmup_step = 400
    num_cycles = 1
    seed = 42
    cv = StratifiedKFold(n_splits=5, shuffle=True)  # StratifiedKFold(n_splits=5, shuffle=True) | 'groupkfold' | StratifiedGroupKFold(n_splits=5, shuffle=True)
    pretrained_weight = None
    

# タグの値を指定    
if cfg.ner_label_name == 'ner_label_bio':
    cfg.tag_values = ['O', 'B-NER', 'I-NER', 'PAD']
elif cfg.ner_label_name == 'ner_label_bioes':
    cfg.tag_values = ['O', 'B-NER', 'I-NER', 'E-NER', 'S-NER', 'PAD']

In [ ]:
cvs = [StratifiedKFold(n_splits=5, shuffle=True)]
folds = [0, 1, 2, 3, 4]

for cv, f in itertools.product(cvs, folds):
    cfg.cv = cv
    cfg.fold = f
    train_fold(cfg)